# 04 - Model Evaluation & Interpretability

This notebook provides:
1. Final test set evaluation (dollar-scale via inverse log transform)
2. Feature importance analysis (all tree-based models)
3. SHAP explanations (global and local)
4. Prediction error analysis
5. Business logic validation checks
6. Model recommendations

In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    print("SHAP not installed. Run: pip install shap")
    SHAP_AVAILABLE = False

plt.style.use('seaborn-v0_8-whitegrid')

## 1. Load Models and Data

In [ ]:
# Load test data (log-scale targets for prediction, original for evaluation)
X_test = pd.read_csv('../data/processed/X_test.csv')
X_test_scaled = pd.read_csv('../data/processed/X_test_scaled.csv')
y_test = pd.read_csv('../data/processed/y_test.csv').squeeze()           # log-scale
y_test_original = pd.read_csv('../data/processed/y_test_original.csv').squeeze()  # dollar-scale

# Load validation data
X_val = pd.read_csv('../data/processed/X_val.csv')
X_val_scaled = pd.read_csv('../data/processed/X_val_scaled.csv')
y_val = pd.read_csv('../data/processed/y_val.csv').squeeze()
y_val_original = pd.read_csv('../data/processed/y_val_original.csv').squeeze()

# Load all models
lr_model = joblib.load('../models/linear_regression_model.joblib')
ridge_model = joblib.load('../models/ridge_model.joblib')
lasso_model = joblib.load('../models/lasso_model.joblib')
dt_model = joblib.load('../models/decision_tree_model.joblib')
rf_model = joblib.load('../models/random_forest_model.joblib')
gb_model = joblib.load('../models/gradient_boosting_model.joblib')

# Load comparison results
results_df = pd.read_csv('../models/model_comparison.csv')
print("Model Comparison (from validation set):")
print(results_df[['model', 'MAE', 'RMSE', 'R2']].to_string(index=False))

## 2. Prediction Error Analysis

In [ ]:
# Predictions on test set — predict in log-space then inverse-transform to dollars
def predict_dollars(model, X):
    """Predict in log-space and convert back to dollars."""
    return np.expm1(model.predict(X))

# Best tree model for detailed analysis
best_tree_model = rf_model
best_tree_name = "Random Forest"

y_pred_tree = predict_dollars(best_tree_model, X_test)
y_pred_ridge = predict_dollars(ridge_model, X_test_scaled)
y_pred_gb = predict_dollars(gb_model, X_test)

# Use original dollar-scale actuals
y_actual = y_test_original.values

# Calculate errors in dollar terms
errors_tree = y_actual - y_pred_tree
errors_ridge = y_actual - y_pred_ridge
errors_gb = y_actual - y_pred_gb

# Print test-set metrics
print("TEST SET EVALUATION (dollar scale)")
print("="*60)
for name, y_pred in [("Ridge", y_pred_ridge), (best_tree_name, y_pred_tree), ("Gradient Boosting", y_pred_gb)]:
    mae = mean_absolute_error(y_actual, y_pred)
    rmse = np.sqrt(mean_squared_error(y_actual, y_pred))
    r2 = r2_score(y_actual, y_pred)
    print(f"  {name:20s}  MAE=${mae:>10,.0f}  RMSE=${rmse:>10,.0f}  R²={r2:.4f}")

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Error distributions (dollar scale)
axes[0, 0].hist(errors_ridge, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(0, color='red', linestyle='--')
axes[0, 0].set_xlabel('Prediction Error ($)')
axes[0, 0].set_title('Ridge Regression - Error Distribution')

axes[0, 1].hist(errors_tree, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].axvline(0, color='red', linestyle='--')
axes[0, 1].set_xlabel('Prediction Error ($)')
axes[0, 1].set_title(f'{best_tree_name} - Error Distribution')

axes[0, 2].hist(errors_gb, bins=50, edgecolor='black', alpha=0.7)
axes[0, 2].axvline(0, color='red', linestyle='--')
axes[0, 2].set_xlabel('Prediction Error ($)')
axes[0, 2].set_title('Gradient Boosting - Error Distribution')

# Predicted vs Actual (dollar scale)
axes[1, 0].scatter(y_actual, y_pred_ridge, alpha=0.3)
axes[1, 0].plot([y_actual.min(), y_actual.max()], [y_actual.min(), y_actual.max()], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual Salary ($)')
axes[1, 0].set_ylabel('Predicted Salary ($)')
axes[1, 0].set_title('Ridge - Predicted vs Actual')

axes[1, 1].scatter(y_actual, y_pred_tree, alpha=0.3)
axes[1, 1].plot([y_actual.min(), y_actual.max()], [y_actual.min(), y_actual.max()], 'r--', lw=2)
axes[1, 1].set_xlabel('Actual Salary ($)')
axes[1, 1].set_ylabel('Predicted Salary ($)')
axes[1, 1].set_title(f'{best_tree_name} - Predicted vs Actual')

axes[1, 2].scatter(y_actual, y_pred_gb, alpha=0.3)
axes[1, 2].plot([y_actual.min(), y_actual.max()], [y_actual.min(), y_actual.max()], 'r--', lw=2)
axes[1, 2].set_xlabel('Actual Salary ($)')
axes[1, 2].set_ylabel('Predicted Salary ($)')
axes[1, 2].set_title('Gradient Boosting - Predicted vs Actual')

plt.tight_layout()
plt.show()

## 3. Feature Importance (Tree-Based Models)

In [ ]:
# Random Forest feature importance
rf_importance = pd.DataFrame({
    'feature': X_test.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 20 Features (Random Forest):")
print(rf_importance.head(20).to_string(index=False))

In [ ]:
# Visualize top 20 features
plt.figure(figsize=(10, 8))
top_20 = rf_importance.head(20)
plt.barh(top_20['feature'], top_20['importance'])
plt.xlabel('Feature Importance')
plt.title('Top 20 Features - Random Forest')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Decision Tree feature importance
dt_importance = pd.DataFrame({
    'feature': X_test.columns,
    'importance': dt_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
top_20_dt = dt_importance.head(20)
plt.barh(top_20_dt['feature'], top_20_dt['importance'])
plt.xlabel('Feature Importance')
plt.title('Top 20 Features - Decision Tree')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Gradient Boosting feature importance
gb_importance = pd.DataFrame({
    'feature': X_test.columns,
    'importance': gb_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
top_20_gb = gb_importance.head(20)
plt.barh(top_20_gb['feature'], top_20_gb['importance'])
plt.xlabel('Feature Importance')
plt.title('Top 20 Features - Gradient Boosting')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 4. Lasso Coefficients (Feature Selection)

In [ ]:
# Lasso coefficients (non-zero = selected features)
lasso_coefs = pd.DataFrame({
    'feature': X_test.columns,
    'coefficient': lasso_model.coef_
})
lasso_coefs['abs_coef'] = lasso_coefs['coefficient'].abs()
lasso_coefs = lasso_coefs.sort_values('abs_coef', ascending=False)

print(f"Non-zero coefficients: {(lasso_coefs['coefficient'] != 0).sum()} / {len(lasso_coefs)}")
print("\nTop 20 Features by Absolute Coefficient (Lasso):")
print(lasso_coefs.head(20)[['feature', 'coefficient']].to_string(index=False))

In [ ]:
# Visualize top Lasso coefficients
top_20_lasso = lasso_coefs.head(20)

plt.figure(figsize=(10, 8))
colors = ['green' if c > 0 else 'red' for c in top_20_lasso['coefficient']]
plt.barh(top_20_lasso['feature'], top_20_lasso['coefficient'], color=colors)
plt.xlabel('Coefficient')
plt.title('Top 20 Features - Lasso Coefficients\n(Green = positive impact, Red = negative impact)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 5. SHAP Analysis

In [ ]:
if SHAP_AVAILABLE:
    print("Computing SHAP values (this may take a few minutes)...")
    
    # Use a sample for faster computation
    sample_size = min(500, len(X_test))
    X_sample = X_test.sample(n=sample_size, random_state=42)
    
    # Create SHAP explainer for Random Forest
    explainer_rf = shap.TreeExplainer(rf_model)
    shap_values_rf = explainer_rf.shap_values(X_sample)
    
    print("SHAP values computed.")

In [ ]:
if SHAP_AVAILABLE:
    # SHAP Summary Plot (Global Explanation)
    plt.figure(figsize=(12, 10))
    shap.summary_plot(shap_values_rf, X_sample, plot_type="bar", max_display=20, show=False)
    plt.title('SHAP Feature Importance (Random Forest)')
    plt.tight_layout()
    plt.show()

In [ ]:
if SHAP_AVAILABLE:
    # SHAP Summary Plot with direction
    plt.figure(figsize=(12, 10))
    shap.summary_plot(shap_values_rf, X_sample, max_display=20, show=False)
    plt.title('SHAP Summary Plot - Feature Impact Direction')
    plt.tight_layout()
    plt.show()

In [ ]:
if SHAP_AVAILABLE:
    # Local Explanation - Waterfall plot for a single prediction
    print("\nLocal Explanation - Example Prediction")
    print("="*60)
    
    # Select a sample
    idx = 0
    sample = X_sample.iloc[idx:idx+1]
    actual = y_test.iloc[X_sample.index[idx]]
    predicted = rf_model.predict(sample)[0]
    
    print(f"Actual Salary: ${actual:,.2f}")
    print(f"Predicted Salary: ${predicted:,.2f}")
    print(f"Error: ${actual - predicted:,.2f}")
    
    # Waterfall plot
    shap.plots.waterfall(shap.Explanation(
        values=shap_values_rf[idx],
        base_values=explainer_rf.expected_value,
        data=X_sample.iloc[idx],
        feature_names=X_sample.columns.tolist()
    ), max_display=15, show=True)

## 6. Error Analysis by Category

In [ ]:
# Analyze errors by salary range (using dollar-scale values)
error_analysis = pd.DataFrame({
    'actual': y_actual,
    'predicted': y_pred_tree,
    'error': errors_tree,
    'abs_error': np.abs(errors_tree)
})

# Bin salaries
error_analysis['salary_bin'] = pd.cut(error_analysis['actual'], 
                                       bins=[0, 75000, 125000, 175000, 500000],
                                       labels=['<75K', '75K-125K', '125K-175K', '>175K'])

# MAE by salary range
mae_by_range = error_analysis.groupby('salary_bin')['abs_error'].agg(['mean', 'std', 'count'])
mae_by_range.columns = ['MAE', 'Std', 'Count']
print(f"MAE by Salary Range ({best_tree_name}):")
print(mae_by_range)

In [ ]:
# Visualize MAE by salary range
plt.figure(figsize=(10, 6))
mae_by_range['MAE'].plot(kind='bar')
plt.xlabel('Salary Range')
plt.ylabel('Mean Absolute Error ($)')
plt.title(f'Prediction Error by Salary Range ({best_tree_name})')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7. Business Logic Validation

Sanity-check that models produce economically sensible predictions:
- **Test 1**: Increasing `years_experience` (0→19) should monotonically increase predicted salary
- **Test 2**: Increasing `experience_level_enc` (EN→MI→SE→EX) should increase predicted salary

In [ ]:
# Build a baseline row from median of training data
baseline_row = X_test.median()

# Models to test — tuple of (name, model, uses_scaled_features)
models_to_check = [
    ('Linear Regression', lr_model, True),
    ('Ridge', ridge_model, True),
    ('Lasso', lasso_model, True),
    ('Decision Tree', dt_model, False),
    ('Random Forest', rf_model, False),
    ('Gradient Boosting', gb_model, False),
]

# Load scaler for scaled baseline
artifacts = joblib.load('../data/processed/preprocessing_artifacts.joblib')
scaler = artifacts['scaler']
cols_to_scale = artifacts['cols_to_scale']

def make_row(base, overrides, scaled=False):
    """Create a single-row DataFrame with overrides applied."""
    row = base.copy()
    for k, v in overrides.items():
        row[k] = v
    df = pd.DataFrame([row])
    if scaled:
        df_scaled = df.copy()
        df_scaled[cols_to_scale] = scaler.transform(df[cols_to_scale])
        return df_scaled
    return df

# ---- Test 1: Vary years_experience 0→19 ----
years_range = list(range(0, 20))
test1_results = {}

for name, model, uses_scaled in models_to_check:
    preds = []
    for yr in years_range:
        row = make_row(baseline_row, {'years_experience': yr}, scaled=uses_scaled)
        pred_dollar = np.expm1(model.predict(row))[0]
        preds.append(pred_dollar)
    test1_results[name] = preds

# ---- Test 2: Vary experience_level_enc EN(0)→MI(1)→SE(2)→EX(3) ----
exp_levels = [0, 1, 2, 3]
exp_labels = ['EN', 'MI', 'SE', 'EX']
test2_results = {}

for name, model, uses_scaled in models_to_check:
    preds = []
    for lvl in exp_levels:
        row = make_row(baseline_row, {'experience_level_enc': lvl}, scaled=uses_scaled)
        pred_dollar = np.expm1(model.predict(row))[0]
        preds.append(pred_dollar)
    test2_results[name] = preds

# ---- Check monotonicity ----
def is_monotonic_increasing(values):
    return all(values[i] <= values[i+1] for i in range(len(values) - 1))

print("="*70)
print("BUSINESS LOGIC VALIDATION RESULTS")
print("="*70)

bl_results = []
for name in [m[0] for m in models_to_check]:
    t1_pass = is_monotonic_increasing(test1_results[name])
    t2_pass = is_monotonic_increasing(test2_results[name])
    status1 = "PASS" if t1_pass else "FAIL"
    status2 = "PASS" if t2_pass else "FAIL"
    bl_results.append({'Model': name, 'Test1_YearsExp': status1, 'Test2_ExpLevel': status2})
    print(f"  {name:25s}  years_exp monotonic: {status1}  |  exp_level monotonic: {status2}")

bl_df = pd.DataFrame(bl_results)
print("\n" + bl_df.to_string(index=False))

In [ ]:
# ---- Visualize Business Logic Tests ----

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Test 1: Years of experience line plot
ax1 = axes[0]
for name in [m[0] for m in models_to_check]:
    ax1.plot(years_range, test1_results[name], marker='o', markersize=3, label=name)
ax1.set_xlabel('Years of Experience')
ax1.set_ylabel('Predicted Salary ($)')
ax1.set_title('Test 1: Predicted Salary vs Years of Experience')
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3)

# Test 2: Experience level bar plot
ax2 = axes[1]
x_pos = np.arange(len(exp_labels))
width = 0.1
for i, name in enumerate([m[0] for m in models_to_check]):
    offset = (i - len(models_to_check) / 2) * width
    ax2.bar(x_pos + offset, test2_results[name], width, label=name)
ax2.set_xlabel('Experience Level')
ax2.set_ylabel('Predicted Salary ($)')
ax2.set_title('Test 2: Predicted Salary vs Experience Level')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(exp_labels)
ax2.legend(fontsize=8)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print summary
n_pass_t1 = sum(1 for r in bl_results if r['Test1_YearsExp'] == 'PASS')
n_pass_t2 = sum(1 for r in bl_results if r['Test2_ExpLevel'] == 'PASS')
n_total = len(bl_results)
print(f"\nTest 1 (years_experience monotonic): {n_pass_t1}/{n_total} models passed")
print(f"Test 2 (experience_level monotonic):  {n_pass_t2}/{n_total} models passed")

## 8. Final Summary & Recommendations

In [ ]:
print("="*70)
print("FINAL MODEL EVALUATION SUMMARY")
print("="*70)

print("\n1. MODEL PERFORMANCE RANKING:")
print("-"*40)
for i, row in results_df.iterrows():
    print(f"   {i+1}. {row['model']}: MAE=${row['MAE']:,.0f}, R²={row['R2']:.4f}")

print("\n2. KEY FINDINGS:")
print("-"*40)
print("   - Tree-based models (Random Forest, Gradient Boosting) typically outperform linear models")
print("   - Log-transform of salary + inverse-transform evaluation ensures dollar-scale metrics")
print("   - Top predictive features include:")
print("     * Experience level and years of experience")
print("     * Job title")
print("     * Company location (country)")
print("     * Technical skills (especially cloud/ML skills)")

print("\n3. BUSINESS LOGIC VALIDATION:")
print("-"*40)
n_pass_t1 = sum(1 for r in bl_results if r['Test1_YearsExp'] == 'PASS')
n_pass_t2 = sum(1 for r in bl_results if r['Test2_ExpLevel'] == 'PASS')
n_total = len(bl_results)
print(f"   - Test 1 (salary increases with years_experience): {n_pass_t1}/{n_total} models passed")
print(f"   - Test 2 (salary increases with experience_level): {n_pass_t2}/{n_total} models passed")
for r in bl_results:
    flag = " " if r['Test1_YearsExp'] == 'PASS' and r['Test2_ExpLevel'] == 'PASS' else " *"
    print(f"     {r['Model']:25s} T1={r['Test1_YearsExp']}  T2={r['Test2_ExpLevel']}{flag}")

print("\n4. RECOMMENDATIONS:")
print("-"*40)
best_model = results_df.iloc[0]['model']
print(f"   - Use {best_model} for production deployment")
print("   - Consider ensemble methods for improved robustness")
print("   - Retrain periodically with updated salary data")
print("   - For interpretability needs, use SHAP explanations")
print("   - Models that fail business logic checks may need monotonic constraints")

print("\n5. LIMITATIONS:")
print("-"*40)
print("   - Dataset is synthetically generated")
print("   - Real-world salary prediction requires verified data sources")
print("   - Model should be validated on actual job market data before production use")

In [ ]:
# Save evaluation artifacts
import os
os.makedirs('../outputs', exist_ok=True)

# Save feature importance
rf_importance.to_csv('../outputs/feature_importance_rf.csv', index=False)
lasso_coefs.to_csv('../outputs/lasso_coefficients.csv', index=False)

# Save business logic results
bl_df.to_csv('../outputs/business_logic_results.csv', index=False)

# Save DT and GB feature importance
dt_importance.to_csv('../outputs/feature_importance_dt.csv', index=False)
gb_importance.to_csv('../outputs/feature_importance_gb.csv', index=False)

print("Evaluation artifacts saved to ../outputs/")
print("  - feature_importance_rf.csv")
print("  - feature_importance_dt.csv")
print("  - feature_importance_gb.csv")
print("  - lasso_coefficients.csv")
print("  - business_logic_results.csv")